In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path

import git

repo = git.Repo(Path(".").absolute(), search_parent_directories=True)
ROOT = Path(repo.working_tree_dir)
SRC = ROOT / "src"

In [ ]:
cd $SRC

# Sentence Representation

In [ ]:
import src.nn as nn

In [ ]:
get_candidate_sents = nn.get_candidate_sents
gen_bvs_i2b = nn.gen_bvs_i2b

In [ ]:
original_unigram_sents = sr.ngram_sents(sents, 1, markers=False)
original_bigram_sents = sr.ngram_sents(sents, 2, markers=True)
original_trigram_sents = sr.ngram_sents(sents, 3, markers=True)

unigram_sent_vecs = sr.ngram_sent_vecs(
    original_unigram_sents, disc.disc, word_vecs, word2index, 1
)
trigram_sent_vecs = sr.ngram_sent_vecs(
    original_trigram_sents, disc.disc, word_vecs, word2index, 3
)

## Reconstruct Unigrams

### Reduce the Size of the Vocabulary

The first 40,000 words close to the sentence cover in 99% of the cases all the unigrams in the sentence. (Evaluated on the validation set).

In [ ]:
word_vecs_full = pickle.load(open(WORD_VECTORS, "rb"))
word2index_full = pickle.load(open(WORD2INDEX, "rb"))
word_vecs = normalize(word_vecs_full[[word2index_full[word] for word in word2index]])

In [ ]:
i = 100
TOP_N = 40000
indices = np.argsort(word_vecs @ unigram_sent_vecs[i])[::-1][:TOP_N]
len(set(original_unigram_sents[i]) & set(index2word[index] for index in indices)) / len(
    set(original_unigram_sents[i])
)

In [ ]:
# TOP_N = 40000

# reconstructed_unigram_sents = []
# for unigram_sent_vec in tqdm(unigram_sent_vecs):
#     indices = np.argsort(word_vecs @ unigram_sent_vec)[::-1][:TOP_N]
#     words = [index2word[index] for index in indices]
#     reconstructed_unigram_sents.append(
#         sr.reconstruct(
#             unigram_sent_vec, word_vecs[indices], dict(enumerate(words)), solver="bp"
#         )
#     )

### Serialize

#### Save

In [ ]:
# file_name = sr.make_file_name(1, sents)
# with open(RESULTS / file_name, "wb") as f:
#     pickle.dump(reconstructed_unigram_sents, f)

#### Load

In [ ]:
file_name = sr.make_file_name(1, sents)

if (RESULTS / file_name).exists():
    with open(RESULTS / file_name, "rb") as f:
        reconstructed_unigram_sents = pickle.load(f)

### Evaluation

In [ ]:
sr.accuracy(original_unigram_sents, reconstructed_unigram_sents)

In [ ]:
# for i in range(len(original_unigram_sents)):
#     if original_unigram_sents[i] != reconstructed_unigram_sents[i]:
#         print(i)
#         print(original_unigram_sents[i] - reconstructed_unigram_sents[i])
#         print(reconstructed_unigram_sents[i] - original_unigram_sents[i])
#         print("==================")